---
title : "Fragility Curve uit een cache"
execute:
    output: asis
---

Voor het vervangen van een fragility curve door een eerder berekende curve maken we gebruik van een `cache`.<br>
Dit is niks anders dat een set aan voorberekende curves. <br>
De logica om te bepalen welke curve uit de cache moet worden gebruikt, moet in de python code worden geïmplementeerd.<br>
De `FragilityCurve` class biedt hiervoor de methode `load_effect_from_data_adapter`.

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt

In [ ]:
from toolbox_continu_inzicht.base.data_adapter import Config, DataAdapter

path = Path.cwd() / "data_sets"
config = Config(config_path=path / "test_fragility_curve_from_cache.yaml")
config.lees_config()
data_adapter = DataAdapter(config=config)

De volgende configuratie wordt gebruikt voor het maken van een GEKB curve
```yaml
GlobalVariables:
    rootdir: "data_sets"
    moments: [-24,0,24,48]

DataAdapter:
    default_options:
        csv:
            sep: ","
    fragility_curve_initial:
        type: csv
        file: "fragility_curve_initial.csv"
    fragility_curve_cache_1:
        type: csv
        file: "fragility_curve_cache_1.csv"
    fragility_curve_cache_2:
        type: csv
        file: "fragility_curve_cache_2.csv"
    fragility_curve_cache_3:
        type: csv
        file: "fragility_curve_cache_3.csv"
    fragility_curve_cache_4:
        type: csv
        file: "fragility_curve_cache_4.csv"
```

Er zijn drie verschillende data bestanden nodig.<br>
Ten eerst een table met de informatie over de helling (slope) waarvoor de curve wordt afgeleid

In [ ]:
data_adapter.input("fragility_curve_initial")

In [ ]:
fig, ax = plt.subplots()
data_adapter.input("fragility_curve_initial").set_index("hydraulicload").plot(ax=ax)
ax.set_ylabel("Failure probability")
for i in range(1, 4):
    data_adapter.input(f"fragility_curve_cache_{i}").drop(
        columns=["Unnamed: 0"]
    ).set_index("hydraulicload").plot(ax=ax, label=f"Cache {i}", alpha=0.15)
ax.legend()
ax.set_xlim(13, 16)

### Door met voorbeeld uit 3